# Evaluate trained model

In [1]:
%load_ext autoreload
%autoreload 1

In [38]:
import os
import cv2

import sys
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image


try:
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box
except ImportError:
    # This part is only required to run the notebook
    # directory when the module itself is not installed.
    #
    # If you have the module installed, just use "import labinet..."
    import os
    import inspect
    # the .travis.yml is coded so that we execute tests from within test subdir. Relative to 'test' the .py is found in ../source/dev
    cmd_subfolder = os.path.realpath(os.path.abspath(os.path.join(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
    if cmd_subfolder not in sys.path:
        sys.path.insert(0, cmd_subfolder)
    from labinet.io_util import load_label_map
    from labinet.io_util import load_model
    import labinet.object_detect
    import labinet.box

%aimport labinet.io_util
%aimport labinet.object_detect
%aimport labinet.box

%matplotlib inline

## change path, model name etc to your needs

In [3]:
MODEL_NAME = 'inference_graph'   # the result from Step 6 Export Inference Graph
IMAGE_PATH = 'images'
IMAGE_NAME = '01-20190414173244-01.jpg'   # cmdline arg
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
CWD_PATH = os.path.join(os.getcwd(),'..')  # should become gitbase
NUM_CLASSES = 1

In [4]:
# model
PATH_TO_MODEL = os.path.join(CWD_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
# label map
LABEL_MAP = os.path.join(CWD_PATH, 'data', 'object-detection.pbtxt')
# path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH, IMAGE_PATH, IMAGE_NAME)

## Load (frozen) Model into Memory

In [5]:
detection_graph = load_model(PATH_TO_MODEL)
#sess = tf.Session(graph=detection_graph)
   

### load labels 

In [6]:
categories, category_index = load_label_map(LABEL_MAP, NUM_CLASSES)

## Run Inference : Object Detection 
_load the image and run on it the object detection_

In [7]:
# load image
image =  Image.open(PATH_TO_IMAGE)
#imshow(np.asarray(image))

# inference
output_dict = labinet.object_detect.run_inference_for_single_image(image, detection_graph)

### Visualize result

In [13]:
image_with_boxes = labinet.object_detect.visualize_boxes_after_detection(image, output_dict, category_index)
# All the results have been drawn on image. Now display the image.
plt.figure(figsize=IMAGE_SIZE)
#plt.imshow(image_with_boxes)
print(IMAGE_NAME)


01-20190414173244-01.jpg


<Figure size 864x576 with 0 Axes>

## Write Result as XML
_use the xml file to laod image in labelImg and check correct boxing_

In [40]:
boxes = output_dict['detection_boxes']
scores = output_dict['detection_scores']
boxes_to_use = labinet.object_detect.get_boxes_to_use(boxes, scores)

In [17]:
# we need the box' coordinates relative to the image size
boxes_normed = []
for box in boxes_to_use:
    box_normed = labinet.box.get_normalized_coordinates(box, image.size)
    boxes_normed.append(box_normed)

In [54]:
%aimport labinet.box

xml = labinet.box.box_to_labelimg_xml(IMAGE_NAME, image.size, boxes_normed, imagepath=PATH_TO_IMAGE)
ET.dump(xml)
xmlfname = PATH_TO_IMAGE[:-4]+".xml"
xml.write(xmlfname)

<annotation><folder>images</folder><filename>01-20190414173244-01.jpg</filename><path>C:\Users\Carsten\Documents\python\object_detection\labinet\notebook\..\images\01-20190414173244-01.jpg</path><source><database>Unknown</database></source><size><width>480</width><height>640</height><depth>3</depth></size><segmented>0</segmented><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>124</xmin><xmax>307</xmax><ymin>426</ymin><ymax>591</ymax></bndbox></object><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>157</xmin><xmax>366</xmax><ymin>456</ymin><ymax>585</ymax></bndbox></object><object><name>dog</name><pose>Unspecified</pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>214</xmin><xmax>381</xmax><ymin>463</ymin><ymax>575</ymax></bndbox></object></annotation>


## Do inference an all new files


In [60]:
NEW_IMAGE_PATH = os.path.join(CWD_PATH, 'new_images')
files = os.listdir(NEW_IMAGE_PATH)

In [62]:
for fname in files:
    if fname.endswith('jpg'):
        image_file = os.path.join(NEW_IMAGE_PATH, fname)
        print(f'inference on: {fname}')
        image =  Image.open(image_file)
        output_dict = labinet.object_detect.run_inference_for_single_image(image, detection_graph)
        boxes = output_dict['detection_boxes']
        scores = output_dict['detection_scores']
        boxes_to_use = labinet.object_detect.get_boxes_to_use(boxes, scores)
        boxes_normed = []
        print(f'found {len(boxes_to_use)} objects')
        for box in boxes_to_use:
            box_normed = labinet.box.get_normalized_coordinates(box, image.size)
            boxes_normed.append(box_normed)

        xml = labinet.box.box_to_labelimg_xml(fname, image.size, boxes_normed, imagepath=image_file)
        xmlfname = image_file[:-4]+".xml"
        xml.write(xmlfname)


inference on: 01-20190414173201-00.jpg
found 0 objects
inference on: 01-20190414173202-00.jpg
found 0 objects
inference on: 01-20190414173205-00.jpg
found 1 objects
inference on: 01-20190414173206-00.jpg
found 0 objects
inference on: 01-20190414173207-00.jpg
found 0 objects
inference on: 01-20190414173208-00.jpg
found 0 objects
inference on: 01-20190414173230-00.jpg
found 2 objects
inference on: 01-20190414173231-00.jpg
found 1 objects
inference on: 01-20190414173232-00.jpg
found 1 objects
inference on: 01-20190414173233-00.jpg
found 2 objects
inference on: 01-20190414173234-00.jpg
found 0 objects
inference on: 01-20190414173236-00.jpg
found 2 objects
inference on: 01-20190414173237-00.jpg
found 2 objects
inference on: 01-20190414173238-00.jpg
found 1 objects
inference on: 01-20190414173240-00.jpg
found 2 objects
inference on: 01-20190414173241-00.jpg
found 1 objects
inference on: 01-20190414173242-00.jpg
found 1 objects
inference on: 01-20190414173243-00.jpg
found 1 objects
inference 

found 0 objects
inference on: 02-20190414174220-00.jpg
found 0 objects
inference on: 02-20190414174221-00.jpg
found 0 objects
inference on: 02-20190414174222-00.jpg
found 0 objects
inference on: 02-20190414174247-00.jpg
found 0 objects
inference on: 02-20190414174248-00.jpg
found 0 objects
inference on: 02-20190414174249-00.jpg
found 1 objects
inference on: 02-20190414174255-00.jpg
found 0 objects
inference on: 02-20190414174300-00.jpg
found 1 objects
inference on: 02-20190414174301-00.jpg
found 0 objects
inference on: 02-20190414174302-00.jpg
found 0 objects
inference on: 02-20190414174308-00.jpg
found 0 objects
inference on: 02-20190414174346-00.jpg
found 0 objects
inference on: 02-20190414174352-00.jpg
found 0 objects
inference on: 02-20190414174353-00.jpg
found 1 objects
inference on: 02-20190414174355-00.jpg
found 2 objects
inference on: 02-20190414174358-00.jpg
found 2 objects
inference on: 02-20190414174400-00.jpg
found 2 objects
inference on: 02-20190414174403-00.jpg
found 1 ob

KeyboardInterrupt: 